In [134]:
# Import libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

In [135]:
def clean_names(names):
    return names.str.lower().str.replace(' ', '').str.replace('-','')

## Top 100 mineable coins

In [136]:
# Top 100 mineable coins
url = 'https://coinmarketcap.com/coins/views/filter-non-mineable/'

r = requests.get(url) 

page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')
table = soup.find_all('div', class_='cmc-table__table-wrapper-outer')

In [137]:
# Get column names
column_names = []
for header in table[0].find_all('th'):
    column_names.append(str(header.string))
# Drop graphs
column_names.pop()
column_names.pop()

data = []
for row in table[2].find_all('tr')[1:]:
    new_row = []
    for entry in row:
        new_row.append(entry.text)
    new_row.pop()
    new_row.pop()
    data.append(new_row)
mineable_100 = pd.DataFrame(data, columns = column_names)
mineable_100 = df.set_index('Name')
mineable_100 = mineable_100.set_index(clean_names(mineable_100.index))

## What to mine, nethash

### ASIC

In [138]:
url = 'https://whattomine.com/asic.json'

r = requests.get(url) 
data = r.json()

whattomine_asic = pd.DataFrame(data)
# normalize + clean names
whattomine_asic = pd.json_normalize(whattomine_asic['coins']).set_index(clean_names(whattomine_data.index.str.lower()))
whattomine_asic['block_time'] = pd.to_numeric(whattomine_asic['block_time'])
whattomine_asic['nethash'] = pd.to_numeric(whattomine_asic['nethash'])
# Drop some columns
whattomine_asic = whattomine_asic[['tag','algorithm', 'block_time', 'difficulty', 'nethash']]
whattomine_asic[:8]

tag    algorithm  block_time    difficulty       nethash
actinium        ACM       Lyra2z       151.0  3.603520e+03  1.024967e+11
axe             AXE          X11       158.0  1.003422e+06  2.727636e+13
bitcoin         BTC      SHA-256       659.0  1.655292e+13  1.078820e+20
bitcoincashabc  BCH      SHA-256       610.0  4.839482e+11  3.407445e+18
bitcoindiamond  BCD          BCD       677.0  3.632004e+06  2.304186e+13
bitcoinsv       BSV      SHA-256       617.0  3.481478e+11  2.423474e+18
bytecoin        BCN  CryptoNight       122.0  2.116895e+10  1.735160e+08
dgbqubit        DGB        Qubit        75.0  6.198668e+06  3.549743e+14

### GPU

In [139]:
url = 'https://whattomine.com/coins.json'

r = requests.get(url) 
data = r.json()

whattomine_gpu = pd.DataFrame(data)
whattomine_gpu = pd.json_normalize(whattomine_gpu['coins']).set_index(clean_names(whattomine_gpu.index.str.lower()))
whattomine_gpu['block_time'] = pd.to_numeric(whattomine_gpu['block_time'])
whattomine_gpu['nethash'] = pd.to_numeric(whattomine_gpu['nethash'])
# Drop some columns
whattomine_gpu = whattomine_gpu[['tag','algorithm', 'block_time', 'difficulty', 'nethash']]
whattomine_gpu[:4]

tag         algorithm  block_time    difficulty  nethash
aeternity    AE       CuckooCycle       184.0  2.694665e+07   146449
aion       AION  Equihash (210,9)        11.0  1.765081e+07  1604618
anon       ANON      EquihashZero       153.0  3.383884e+05     2211
beam       BEAM        BeamHashII        59.0  1.241485e+08  2104211

## CoinWarz

In [140]:
url = 'https://www.coinwarz.com/v1/api/profitability?apikey=58467a6f2e2d4c65b7ef720b26137453&algo=all'
r = requests.get(url) 
data = r.json()

coinwarz = pd.DataFrame(data)
    
coinwarz = pd.json_normalize(coinwarz['Data'])
coinwarz = coinwarz.set_index(clean_names(coinwarz['CoinName']))

# Drop and rename some columns
coinwarz = coinwarz[['CoinTag', 'Algorithm','BlockTimeInSeconds', 'Difficulty']]
coinwarz = coinwarz.rename(columns={'CoinTag': 'tag', 'Algorithm': 'algorithm','BlockTimeInSeconds':'block_time', 'Difficulty': 'difficulty'})
coinwarz['nethash_scraped'] = 0
coinwarz

tag  algorithm  block_time    difficulty  nethash_scraped
CoinName                                                                    
ethereumclassic    ETC     EtHash          15  1.600628e+14                0
ethereum           ETH     EtHash          15  2.185784e+15                0
horizen            ZEN   Equihash         150  1.081035e+07                0
verge              XVG     Scrypt          30  1.148953e+04                0
cannabiscoin      CANN        X11          42  3.208095e+04                0
zcash              ZEC   Equihash          75  5.441837e+07                0
peercoin           PPC    SHA-256         600  1.600011e+09                0
bitcoincash        BCH    SHA-256         600  4.830042e+11                0
bitcoin            BTC    SHA-256         600  1.655292e+13                0
feathercoin        FTC  NeoScrypt          60  5.389772e+01                0
ravencoin          RVN     X16Rv2          60  4.479893e+05                0
vertcoin           VTC  Lyra2REv3         150  7.440796e+04                0
monacoin          MONA  Lyra2REv2          90  1.362650e+06                0
litecoin           LTC     Scrypt         150  6.364954e+06                0
groestlcoin        GRS    Groestl          60  1.351008e+06                0
monero             XMR    RandomX         120  1.447151e+11                0
florincoin         FLO     Scrypt          40  1.793511e+03                0
gamecredits       GAME     Scrypt          90  6.060408e+03                0
einsteinium       EMC2     Scrypt          60  1.420122e+03                0
dash              DASH        X11         150  1.237686e+08                0
startcoin        START        X11          60  4.768078e+02                0
litecoincash       LCC    SHA-256          75  2.163651e+08                0
digibyte           DGB     Scrypt          15  7.202586e+04                0
dogecoin          DOGE     Scrypt          60  1.930991e+06                0
syscoin            SYS    SHA-256          60  2.470251e+11                0
namecoin           NMC    SHA-256         600  1.469214e+13                0
auroracoin         AUR     Scrypt         300  6.284313e+02                0
novacoin           NVC     Scrypt         450  1.324724e+04                0
viacoin            VIA     Scrypt          24  2.726739e+05                0

# Combining the data

In [171]:
def check_missing(data):
    missing_data = result[data.isnull().any(axis=1)]
    return missing_data.shape[0]

In [141]:
whattomine = whattomine_gpu.append(whattomine_asic)
whattomine = whattomine.rename(columns={'nethash':'nethash_scraped'})
whattomine

tag         algorithm  block_time    difficulty  \
aeternity         AE       CuckooCycle       184.0  2.694665e+07   
aion            AION  Equihash (210,9)        11.0  1.765081e+07   
anon            ANON      EquihashZero       153.0  3.383884e+05   
beam            BEAM        BeamHashII        59.0  1.241485e+08   
bitcash         BITC              X25X        63.0  5.900000e+01   
...              ...               ...         ...           ...   
vergegroestl     XVG       Myr-Groestl       150.0  6.992196e+06   
vergelyra2rev2   XVG         Lyra2REv2       150.0  1.622511e+05   
vergescrypt      XVG            Scrypt       150.0  1.148953e+04   
viacoin          VIA            Scrypt        24.0  2.647808e+05   
zcash            ZEC          Equihash        75.0  5.331957e+07   

                nethash_scraped  
aeternity          1.464490e+05  
aion               1.604618e+06  
anon               2.211000e+03  
beam               2.104211e+06  
bitcash            4.022271e+09  
...                         ...  
vergegroestl       2.002084e+14  
vergelyra2rev2     4.645756e+12  
vergescrypt        3.289811e+11  
viacoin            4.738437e+13  
zcash              5.823919e+09  

[117 rows x 5 columns]

In [172]:
result = mineable_100.join(whattomine, how="left")

67

In [173]:
for coin in coinwarz.index:
    if coin not in whattomine.index.values:
        if coin in result.index:
            result.loc[coin, ['tag','algorithm','block_time','difficulty','nethash_scraped']] = coinwarz.loc[coin]
check_missing(result)

62

In [175]:
result[:10]

#       Market Cap      Price     Volume (24h)  \
Name                                                               
bitcoin           1  $98,046,608,781  $5,366.33  $37,200,850,186   
ethereum          2  $14,119,168,507    $128.24  $13,427,190,349   
bitcoincash       3   $3,156,523,320    $172.20   $3,693,363,953   
litecoin          4   $2,281,325,629     $35.49   $3,433,780,345   
bitcoinsv         5   $2,197,405,214    $119.89   $1,843,497,487   
cardano           6     $700,197,528  $0.027006      $69,761,958   
tron              7     $674,746,961  $0.010119   $1,180,757,757   
monero            8     $656,752,466     $37.57     $118,871,633   
ethereumclassic   9     $552,141,573      $4.75   $1,388,281,248   
dash             10     $446,401,779     $47.59     $490,749,431   

                 Circulating Supply Change (24h)   tag algorithm  block_time  \
Name                                                                           
bitcoin              18,270,687 BTC       -1.30%   BTC   SHA-256    659.0000   
ethereum            110,096,178 ETH       -0.38%   ETH    Ethash     13.3073   
bitcoincash          18,330,438 BCH       -0.03%   BCH   SHA-256    600.0000   
litecoin             64,287,243 LTC       -1.84%   LTC    Scrypt    196.0000   
bitcoinsv            18,327,890 BSV        2.81%   BSV   SHA-256    617.0000   
cardano          25,927,070,538 ADA       -1.45%   NaN       NaN         NaN   
tron             66,682,072,191 TRX        0.82%   NaN       NaN         NaN   
monero               17,482,745 XMR        4.99%   XMR   RandomX    118.0000   
ethereumclassic     116,313,299 ETC       -0.24%   ETC    Ethash     12.8655   
dash                 9,380,755 DASH       -3.04%  DASH       X11    158.0000   

                   difficulty  nethash_scraped  
Name                                            
bitcoin          1.655292e+13     1.078820e+20  
ethereum         2.179376e+15     1.637729e+14  
bitcoincash      4.830042e+11     0.000000e+00  
litecoin         6.365051e+06     1.394780e+14  
bitcoinsv        3.481478e+11     2.423474e+18  
cardano                   NaN              NaN  
tron                      NaN              NaN  
monero           1.451387e+11     1.229989e+09  
ethereumclassic  1.595933e+14     1.240475e+13  
dash             1.322249e+08     3.594314e+15

In [181]:
found_data = result[~result.isnull().any(axis=1)]
found_data

#       Market Cap      Price     Volume (24h)  \
Name                                                               
bitcoin           1  $98,046,608,781  $5,366.33  $37,200,850,186   
ethereum          2  $14,119,168,507    $128.24  $13,427,190,349   
bitcoincash       3   $3,156,523,320    $172.20   $3,693,363,953   
litecoin          4   $2,281,325,629     $35.49   $3,433,780,345   
bitcoinsv         5   $2,197,405,214    $119.89   $1,843,497,487   
monero            8     $656,752,466     $37.57     $118,871,633   
ethereumclassic   9     $552,141,573      $4.75   $1,388,281,248   
dash             10     $446,401,779     $47.59     $490,749,431   
zcash            11     $243,247,669     $25.90     $217,475,929   
dogecoin         12     $211,539,578  $0.001710     $118,995,381   
decred           13     $119,294,925     $11.06      $58,973,438   
bitcoingold      14     $112,071,926      $6.40      $18,395,866   
ravencoin        16      $81,008,734  $0.014161       $9,377,141   
bitcoindiamond   17      $72,914,304  $0.390976       $2,154,565   
monacoin         18      $71,337,749      $1.09       $9,828,646   
horizen          21      $48,024,056      $5.57         $859,977   
digibyte         23      $44,939,955  $0.003477         $707,208   
bytecoin         24      $44,608,500  $0.000242          $12,383   
komodo           26      $38,217,856  $0.321672       $1,588,004   
verge            28      $34,267,540  $0.002114         $845,620   
zcoin            30      $29,057,488      $3.00       $7,351,779   
aeternity        31      $27,996,885  $0.092024       $9,363,362   
aion             32      $24,755,592  $0.061769       $1,990,492   
beam             37      $13,683,484  $0.238200      $18,146,900   
vertcoin         41      $10,689,720  $0.198461         $589,096   
einsteinium      43      $10,170,066  $0.046324       $3,490,856   
groestlcoin      45       $9,383,984  $0.125741       $1,172,250   
syscoin          46       $9,082,855  $0.015633         $209,938   
namecoin         65       $4,172,702  $0.283156               $8   
peercoin         68       $3,512,981  $0.134778          $53,054   
bitcore          71       $3,202,278  $0.181538             $839   
smartcash        72       $3,157,407  $0.002233         $152,570   
litecoincash     79       $2,563,672  $0.003842          $62,456   
viacoin          86       $2,241,985  $0.096782          $42,623   
ubiq             89       $2,181,175  $0.051190           $1,429   
gamecredits      90       $2,133,603  $0.030552           $7,974   
zel              91       $2,121,550  $0.022142       $8,118,176   
axe              97       $1,713,179  $0.325640       $5,057,564   

                   Circulating Supply Change (24h)    tag         algorithm  \
Name                                                                          
bitcoin                18,270,687 BTC       -1.30%    BTC           SHA-256   
ethereum              110,096,178 ETH       -0.38%    ETH            Ethash   
bitcoincash            18,330,438 BCH       -0.03%    BCH           SHA-256   
litecoin               64,287,243 LTC       -1.84%    LTC            Scrypt   
bitcoinsv              18,327,890 BSV        2.81%    BSV           SHA-256   
monero                 17,482,745 XMR        4.99%    XMR           RandomX   
ethereumclassic       116,313,299 ETC       -0.24%    ETC            Ethash   
dash                   9,380,755 DASH       -3.04%   DASH               X11   
zcash                   9,391,356 ZEC       -3.11%    ZEC          Equihash   
dogecoin         123,723,547,816 DOGE       -0.01%   DOGE            Scrypt   
decred                 10,786,831 DCR        7.21%    DCR       Blake (14r)   
bitcoingold            17,513,924 BTG        1.45%    BTG             Zhash   
ravencoin           5,720,410,000 RVN        2.74%    RVN            X16Rv2   
bitcoindiamond        186,492,898 BCD        2.30%    BCD               BCD   
monacoin              65,729,675 

In [183]:
found_data['algorithm'].value_counts()

Scrypt              7
SHA-256             7
Equihash            3
X11                 2
Ethash              2
RandomX             1
X16Rv2              1
Groestl             1
Lyra2REv2           1
MTP                 1
TimeTravel10        1
Blake (14r)         1
Lyra2REv3           1
BCD                 1
BeamHashII          1
Keccak              1
Equihash (210,9)    1
ZelHash             1
Zhash               1
CryptoNight         1
Ubqhash             1
CuckooCycle         1
Name: algorithm, dtype: int64